In [1]:
using Revise

In [2]:
using RigidBodyDynamics
using RigidBodyTreeInspector
using DrakeVisualizer
DrakeVisualizer.any_open_windows() || DrakeVisualizer.new_window()
using LCPSim
using Polyhedra
using CDDLib
using StaticArrays: SVector
using Gurobi
using JuMP
using LearningMPC
using ExplicitQPs
using Plots; gr()
using ProgressMeter

INFO: Recompiling stale cache file /home/rdeits/locomotion/explorations/learning-mpc/packages/lib/v0.6/ExplicitQPs.ji for module ExplicitQPs.


In [5]:
mechanism = parse_urdf(Float64, "cartpole.urdf")
world = root_body(mechanism)

basevis = Visualizer()[:cartpole]
delete!(basevis)
vis = basevis[:robot]
setgeometry!(vis, mechanism, parse_urdf("cartpole.urdf", mechanism))

wall_radius = 1.5
μ = 0.5
walls = [planar_obstacle(default_frame(world), [1., 0, 0.], [-wall_radius, 0, 0.], μ), 
    planar_obstacle(default_frame(world), [-1., 0, 0.], [wall_radius, 0, 0.], μ)]
bounds = SimpleHRepresentation(vcat(eye(3), -eye(3)), vcat([wall_radius + 0.1, 0.5, 2.0], -[-wall_radius - 0.1, -0.5, -0.1]))

for wall in walls
    addgeometry!(basevis[:environment], CDDPolyhedron{3, Float64}(intersect(wall.interior, bounds)))
end

pole = findbody(mechanism, "pole")
env = Environment(
    Dict(pole => ContactEnvironment(
            [Point3D(default_frame(pole), SVector(0., 0, 1))],
            walls)))
# env = Environment{Float64}(Dict())

inspect(mechanism, vis)

Interact.Options{:SelectionSlider,Float64}(11: "input-5" = 0.0 Float64 , "slider_to_cart.1", 0.0, "0.0", Interact.OptionDict(DataStructures.OrderedDict("-3.141592653589793"=>-3.14159,"-3.015928947446201"=>-3.01593,"-2.8902652413026098"=>-2.89027,"-2.764601535159018"=>-2.7646,"-2.6389378290154264"=>-2.63894,"-2.5132741228718345"=>-2.51327,"-2.3876104167282426"=>-2.38761,"-2.261946710584651"=>-2.26195,"-2.1362830044410592"=>-2.13628,"-2.0106192982974678"=>-2.01062…), Dict(-3.01593=>"-3.015928947446201",2.89027=>"2.8902652413026098",1.75929=>"1.7592918860102842",2.63894=>"2.6389378290154264",0.251327=>"0.25132741228718347",-2.51327=>"-2.5132741228718345",1.25664=>"1.2566370614359172",-3.14159=>"-3.141592653589793",1.3823=>"1.382300767579509",2.7646=>"2.764601535159018"…)), Any[], Any[], true, "horizontal")

Interact.Options{:SelectionSlider,Float64}(13: "input-6" = 0.0 Float64 , "cart_to_pole.1", 0.0, "0.0", Interact.OptionDict(DataStructures.OrderedDict("-3.141592653589793"=>-3.14159,"-3.015928947446201"=>-3.01593,"-2.8902652413026098"=>-2.89027,"-2.764601535159018"=>-2.7646,"-2.6389378290154264"=>-2.63894,"-2.5132741228718345"=>-2.51327,"-2.3876104167282426"=>-2.38761,"-2.261946710584651"=>-2.26195,"-2.1362830044410592"=>-2.13628,"-2.0106192982974678"=>-2.01062…), Dict(-3.01593=>"-3.015928947446201",2.89027=>"2.8902652413026098",1.75929=>"1.7592918860102842",2.63894=>"2.6389378290154264",0.251327=>"0.25132741228718347",-2.51327=>"-2.5132741228718345",1.25664=>"1.2566370614359172",-3.14159=>"-3.141592653589793",1.3823=>"1.382300767579509",2.7646=>"2.764601535159018"…)), Any[], Any[], true, "horizontal")

In [6]:
x0 = MechanismState{Float64}(mechanism)
set_velocity!(x0, zeros(num_velocities(x0)))
set_configuration!(x0, findjoint(mechanism, "slider_to_cart"), [0])
set_configuration!(x0, findjoint(mechanism, "cart_to_pole"), [0])
q0 = copy(configuration(x0))
v0 = copy(velocity(x0))
u0 = zeros(num_velocities(x0))
xstar = MechanismState(mechanism, q0, v0)

contacts = Point3D[]
Q = diagm([10, 10, 1, 1])
R = 0.1 * eye(num_velocities(x0))
K, S = LCPSim.ContactLQR.contact_lqr(x0, u0, Q, R, contacts)

controller = x -> begin
    -K * (state_vector(x) - state_vector(xstar)) .+ u0
#     u0
end
Δt = 0.01

# set_velocity!(x0, findjoint(mechanism, "slider_to_cart"), [-2])
set_configuration!(x0, findjoint(mechanism, "cart_to_pole"), [π/4])

results = LCPSim.simulate(x0, controller, env, Δt, 200, GurobiSolver(OutputFlag=0));

In [7]:
playback(vis, results, Δt)

In [18]:
function run_optimization(x0, env, Δt, N; x_nominal=x0, solver=GurobiSolver())
    qstar = copy(configuration(x_nominal))
    vstar = zeros(num_velocities(x_nominal))
    ustar = u0
    
    model = Model(solver=solver)
    @variable model q0[1:num_positions(x0)]
    JuMP.fix.(q0, configuration(x0))
    @variable model v0[1:num_velocities(x0)]
    JuMP.fix.(v0, velocity(x0))
    
    _, results_opt = LCPSim.optimize(MechanismState(x0.mechanism, q0, v0), env, Δt, N, model)
    
    objective = (
        sum(Δt * (r.state.state' * Q * r.state.state + 0.01 * r.input' * R * r.input) for r in results_opt)
       + (results_opt[end].state.state' * S * results_opt[end].state.state)
        )

    for r in results_opt
        for (joint, jrs) in r.joint_contacts
            for joint_result in jrs
                objective += joint_result.λ^2
            end
        end 
    end
    
    @objective model Min objective

    controller = x -> begin
        -K * (state_vector(x) - vcat(qstar, vstar)) .+ ustar
    end
    
    Δt_sim = 0.01
    time_ratio = convert(Int, Δt / Δt_sim)
    results_lqr = LCPSim.simulate(x0, controller, env, Δt_sim, time_ratio * N, GurobiSolver(OutputFlag=0))
    if length(results_lqr[1:time_ratio:end]) == length(results_opt)
        setvalue.(results_opt, results_lqr[1:time_ratio:end])
        ConditionalJuMP.warmstart!(model, false)
    end
    solve(model)
    
    results = getvalue.(results_opt)
    
    # Now fix the binary variables and re-solve to get updated duals
    ConditionalJuMP.warmstart!(model, true)
    @assert sum(model.colCat .== :Bin) == 0 "Model should no longer have any binary variables"
    
    
    # Ensure objective is strictly PD
    nvars = length(model.colCat)
    vars = [Variable(model, i) for i in 1:nvars]
    @objective model Min objective + QuadExpr(vars, vars, [1e-6 for v in vars], AffExpr([], [], 0.0))
    solve(model)
    @time exsol = ExplicitQPs.explicit_solution(model, vcat(q0, v0))
    J = ExplicitQPs.jacobian(exsol, results_opt[1].input)
    
    return results, J
end

run_optimization (generic function with 1 method)

In [ ]:
set_configuration!(x0, q0)
set_velocity!(x0, v0)
# set_configuration!(x0, findjoint(mechanism, "slider_to_cart"), [-1.5])
# set_velocity!(x0, findjoint(mechanism, "slider_to_cart"), [-2])
set_configuration!(x0, findjoint(mechanism, "cart_to_pole"), [π/4])
Δt = 0.01
gap = 1e-3
timelimit = 60
horizon = 30

r_control = []
x_control = MechanismState(mechanism, copy(q0), copy(v0))
controller = x -> begin
    set_velocity!(x_control, velocity(x))
    set_configuration!(x_control, configuration(x))
    settransform!(vis, x_control)
    r_control, J = run_optimization(x_control, env, Δt, horizon; x_nominal=xstar, solver=GurobiSolver(OutputFlag=0, MIPGap=gap, TimeLimit=timelimit))
    u = r_control[1].input
#     for r in r_control
#         set_configuration!(x0, configuration(r.state))
#         settransform!(vis, x0)
#     end
    u
    
    @show u
end

results_mpc = LCPSim.simulate(x0, controller, env, Δt, 200, GurobiSolver(OutputFlag=0));

  4.363216 seconds (48.11 k allocations: 58.454 MiB, 0.38% gc time)
u = [20.0, -0.0]
  4.421810 seconds (48.70 k allocations: 61.474 MiB, 0.44% gc time)
u = [20.0, -0.0]
  3.571783 seconds (48.48 k allocations: 60.405 MiB, 0.56% gc time)
u = [20.0, -0.0]
  3.971821 seconds (47.65 k allocations: 56.208 MiB, 0.42% gc time)
u = [20.0, -0.0]
  4.055573 seconds (48.18 k allocations: 58.662 MiB, 0.42% gc time)
u = [20.0, -0.0]
  4.449723 seconds (48.64 k allocations: 60.840 MiB, 0.83% gc time)
u = [20.0, -0.0]
  4.418557 seconds (48.43 k allocations: 59.781 MiB, 2.77% gc time)
u = [20.0, -0.0]
  4.296134 seconds (48.44 k allocations: 59.645 MiB, 0.55% gc time)
u = [20.0, -0.0]
  3.985802 seconds (48.37 k allocations: 59.227 MiB, 0.69% gc time)
u = [20.0, -0.0]
  4.068390 seconds (48.91 k allocations: 61.781 MiB, 0.49% gc time)
u = [20.0, -0.0]
  4.299180 seconds (48.47 k allocations: 59.445 MiB, 0.48% gc time)
u = [20.0, -0.0]
  4.665217 seconds (48.63 k allocations: 60.080 MiB, 0.49% gc tim

  6.672237 seconds (49.64 k allocations: 64.653 MiB, 0.34% gc time)
u = [-19.7465, -0.0]
  4.677060 seconds (49.38 k allocations: 63.325 MiB, 0.27% gc time)
u = [-11.8574, -0.0]
  4.489020 seconds (49.35 k allocations: 63.179 MiB, 0.75% gc time)
u = [-5.63419, -0.0]
  5.407794 seconds (49.62 k allocations: 64.434 MiB, 10.21% gc time)
u = [-0.347443, -0.0]
  5.741706 seconds (49.34 k allocations: 63.035 MiB, 0.36% gc time)
u = [4.62173, -0.0]
  4.621900 seconds (49.29 k allocations: 62.744 MiB, 0.64% gc time)
u = [9.85308, -0.0]
  4.552996 seconds (49.40 k allocations: 63.255 MiB, 0.55% gc time)
u = [16.0021, -0.0]
  5.464203 seconds (49.34 k allocations: 62.750 MiB, 0.40% gc time)
u = [20.0, -0.0]
  4.411358 seconds (49.54 k allocations: 63.702 MiB, 0.49% gc time)
u = [20.0, -0.0]
  4.315234 seconds (49.31 k allocations: 62.533 MiB, 0.48% gc time)
u = [20.0, -0.0]
  4.354716 seconds (49.32 k allocations: 62.533 MiB, 0.47% gc time)
u = [20.0, -0.0]
  4.399433 seconds (49.29 k allocation

In [16]:
playback(vis, results_mpc, Δt)

LoadError: [91mUndefVarError: results_mpc not defined[39m

In [ ]:
set_configuration!(x0, configuration(results_mpc[end].state))
set_velocity!(x0, velocity(results_mpc[end].state))

r_control = []
x_control = MechanismState(mechanism, copy(q0), copy(v0))
controller = x -> begin
    set_velocity!(x_control, velocity(x))
    set_configuration!(x_control, configuration(x))
    r_control = run_optimization(x_control, env, Δt, horizon; x_nominal=xstar, solver=GurobiSolver(OutputFlag=0, MIPGap=gap, TimeLimit=timelimit))
    u = r_control[1].input
    @show u
end

append!(results_mpc, LCPSim.simulate(x0, controller, env, Δt, 100, GurobiSolver(OutputFlag=0)));